In [0]:
import smtplib
from email.mime.text import MIMEText
import datetime
import time

# Fetch SMTP credentials
smtp_user = dbutils.secrets.get(scope="secretscope_datacapus6", key="smtp-user")
smtp_password = dbutils.secrets.get(scope="secretscope_datacapus6", key="smtp-password")

smtp_server = "smtp.gmail.com"
smtp_port = 587

def send_email(subject, body, retries=3, delay=5):
    sender = smtp_user
    recipients = dbutils.secrets.get(scope="secretscope_datacapus6", key="alert-recipients").split(",")  # Or fetch from secrets for multiple recipients

    # Use HTML format for better readability
    msg = MIMEText(body, "html")
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ", ".join(recipients)

    for attempt in range(retries):
        try:
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls()
                server.login(smtp_user, smtp_password)
                server.sendmail(sender, recipients, msg.as_string())
            print(f"Email sent successfully on attempt {attempt+1}: {subject}")
            return
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("All attempts to send email failed.")

# Capture start time
start_time = datetime.datetime.now()

try:
    print("Running Bronze Layer...")
    dbutils.notebook.run("Bronze_Layer", 600)

    print("Running Silver Layer...")
    dbutils.notebook.run("Silver_Layer", 600)

    print("Running Gold Layer...")
    dbutils.notebook.run("Gold_Layer", 600)

    # Capture end time and duration
    end_time = datetime.datetime.now()
    duration = round((end_time - start_time).total_seconds() / 60, 2)

    # ✅ Success Email Body (HTML)
    body = f"""
    <html>
    <body style="font-family:Arial; color:#333;">
        <h2 style="color:green;">✅ Pipeline Execution Successful</h2>
        <p><b>Start Time:</b> {start_time.strftime('%Y-%m-%d %H:%M:%S')}<br>
        <b>End Time:</b> {end_time.strftime('%Y-%m-%d %H:%M:%S')}<br>
        <b>Duration:</b> {duration} minutes</p>
        <h3>Layers Executed:</h3>
        <ul>
            <li>Bronze Layer</li>
            <li>Silver Layer</li>
            <li>Gold Layer</li>
        </ul>
    </body>
    </html>
    """
    send_email("Medallion Pipeline Execution Successful", body)
    print("Pipeline executed successfully.")

except Exception as ex:
    failure_time = datetime.datetime.now()
    error_msg = str(ex)

    # ❌ Failure Email Body (HTML)
    body = f"""
    <html>
    <body style="font-family:Arial; color:#333;">
        <h2 style="color:red;">❌ Pipeline Execution Failed</h2>
        <p><b>Start Time:</b> {start_time.strftime('%Y-%m-%d %H:%M:%S')}<br>
        <b>Failure Time:</b> {failure_time.strftime('%Y-%m-%d %H:%M:%S')}</p>
        <h3>Error Details:</h3>
        <pre style="background:#f8f8f8; padding:10px; border:1px solid #ccc;">{error_msg}</pre>
    </body>
    </html>
    """
    send_email("Medallion Pipeline Execution FAILED", body)
    print(f"Pipeline failed: {error_msg}")
